# World development indicators data set



## Motivation
In recent climate change conferences developing countries are considered to be big pollutors. Developing economies have energy needs to support their economies and lead to pollution. Developed countries were asked to innovate ways to curb the growing emissions. I belive in climate change and that is why I am interested in this dataset. Climate change impacts all humanity irrespective of the which country is the biggest pollutor. 

## Questions to explore
I want to understand how globalization has shifted energy needs from developed countries to developing countries.  If globalization helped developing economies support their citizens they could be paying a price in pollution which could be impacting the health. 

My hypothesis is globalizaton increased energy consumption for developing economies and decreased the enery consumption of developed countries. 

The second quesion I want to pursue is improvement in quality of life in developing countries as a result of globalization. 
This is very broad question we can answer this with GDP, mortality rate, education and health metrics. 

My hypothesis is globalization improved the quality of life developing countries. 

## Data set
The data set to use for this analysis is [world development indicators](https://datacatalog.worldbank.org/dataset/world-development-indicators). This data has close to 1600 indicators. Energy consumption and infrastructure metrics are tracked on a per country level. More information on the indicators can be found [here](https://data.worldbank.org/indicator).

## License
This data is public and uses CC-BY 4.0 licenses. It is permitted to download, redistribute the data with proper attribution 

## Analysis

I want to consider data from year 2000 onwards. There are enegry indicators for each country in the list. In the total energy consumption the indicator of interest is amount of energy imported. I want to look at energy consumption and energy import for developing countries like India, China and compare that to developed countries like USA and European countries. 

For the second question I am going to use GDP , education, mortality rate indicators for developing economies. I need to figure out how this improvement can be tied to globalization. 




In [2]:
import pandas as pd 

In [3]:
# load data from 
wdidata_df_orig = pd.read_csv('data/WDIData.csv')


In [4]:
# emission columns. There are multiple indicators for this 
greenhouse_gases = 'Total greenhouse gas emissions (kt of CO2 equivalent)'

wdidata_df_orig.head(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN,NaN


In [8]:
ind_df = wdidata_df_orig[(wdidata_df_orig['Indicator Code'] == 'EN.ATM.GHGT.KT.CE') & (wdidata_df_orig['Country Code'] == 'IND')]

In [11]:
pd.melt(ind_df, id_vars=['Country Code', 'Indicator Name'], value_vars=['2000', '2001', '2002'])

,Country Code,Indicator Name,variable,value
0,IND,Total greenhouse gas emissions (kt of CO2 equi...,2000,1885189.00
1,IND,Total greenhouse gas emissions (kt of CO2 equi...,2001,1875600.85
2,IND,Total greenhouse gas emissions (kt of CO2 equi...,2002,1917294.66
